<a href="https://colab.research.google.com/github/C242-PS253-Capstone-Team/ML_Face_Shape_Recognition/blob/rifal/ML_Model_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mssf